In [1]:
import os
import boto3 
import pandas as pd
import scipy.stats

In [23]:
bucket_name = 'advanced-ml-project' # Add your bucket name
file_name = 'sessions_train.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
df_train = pd.read_csv(obj['Body'])
df_train.head()

,Unnamed: 0,session_id,timezone,timezone_offset,previous_sessions_duration,is_user_first_session,is_session,country,region,city,...,locale,os_name,session_index,device_id,user_id_hash,start_dow,start_hour,start_date,create_date,create_session_time
0,0,5558845121177764917,Asia/Manila,28800000.0,25837591,False,True,PH,00,makati,...,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,1,2018-11-15,2018-10-07,39
1,1,18781111175537580,Asia/Manila,28800000.0,11343848,False,True,PH,00,makati,...,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,7,2018-10-11,2018-10-07,4
2,2,1477540082628742048,Asia/Manila,28800000.0,13499724,False,True,PH,11,davao city,...,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,19,2018-10-21,2018-10-07,14
3,3,8184875317380844086,Asia/Manila,28800000.0,32788010,False,True,PH,00,makati,...,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,7,2018-11-20,2018-10-07,44
4,4,4706180700083856343,Asia/Manila,28800000.0,5872534,False,True,PH,11,davao city,...,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,19,2018-10-08,2018-10-07,1


In [ ]:
bucket_name = 'advanced-ml-project' # Add your bucket name
file_name = 'train_feature2.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
train_feature = pd.read_csv(obj['Body'])
train_feature.head()

### Time_stamp

* aggregate features - user-level

In [4]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)

In [5]:
df_train.head()

,session_id,timezone,timezone_offset,previous_sessions_duration,is_user_first_session,is_session,country,region,city,latitude,...,locale,os_name,session_index,device_id,user_id_hash,start_dow,start_hour,start_date,create_date,create_session_time
0,5558845121177764917,Asia/Manila,28800000.0,25837591,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,1,2018-11-15,2018-10-07,39
1,18781111175537580,Asia/Manila,28800000.0,11343848,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,7,2018-10-11,2018-10-07,4
2,1477540082628742048,Asia/Manila,28800000.0,13499724,False,True,PH,11,davao city,7.190708,...,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,19,2018-10-21,2018-10-07,14
3,8184875317380844086,Asia/Manila,28800000.0,32788010,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,7,2018-11-20,2018-10-07,44
4,4706180700083856343,Asia/Manila,28800000.0,5872534,False,True,PH,11,davao city,7.190708,...,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,19,2018-10-08,2018-10-07,1


In [ ]:
pd.isnull(df_train).any()

In [ ]:
train_feature = pd.DataFrame(df_train.user_id_hash.unique(), columns=['user_id_hash'])

In [ ]:
train_feature['most_freq_dow'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').start_dow.agg(lambda x: x.value_counts().index[0]))

In [ ]:
train_feature['most_freq_hour'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').start_hour.agg(lambda x: x.value_counts().index[0]))

In [ ]:
train_feature['median_create_session_time'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').create_session_time.median())

In [ ]:
# seems no much use
def ent(data):
    p_data = data.value_counts()/len(data)  # calculates the probabilities
    # input probabilities to get the entropy
    entropy = scipy.stats.entropy(p_data)
    return entropy

In [ ]:
train_feature['user_entropy'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').session_id.apply(ent))

In [ ]:
train_feature.head()

### session 

* study more on user session properties to get duration, interval and frequency features.

In [ ]:
train_feature['session_count'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').session_index.agg(lambda x: max(x)))

In [ ]:
train_feature['session_durations'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').previous_sessions_duration.sum())

In [ ]:
train_feature['mean_session_durations'] = train_feature.session_durations/train_feature.session_count

In [ ]:
train_feature['mean_session_interval'] = train_feature.median_create_session_time/train_feature.session_count

In [ ]:
import numpy as np
train_feature['mean_session_interval'].replace(np.inf, 0, inplace=True)

In [ ]:
date_info = df_train.groupby('user_id_hash').start_date.nunique()
train_feature['active_date'] = train_feature.user_id_hash.map(date_info)
train_feature.active_date = train_feature.active_date.fillna(train_feature['active_date'].value_counts().index[0])
train_feature['session_per_active_day'] = train_feature.session_count.div(train_feature.active_date)
train_feature['purchase_per_active_day'] = train_feature.purchase_counts.div(train_feature.active_date)

In [ ]:
create_time = df_train.groupby('user_id_hash').create_session_time.max()
train_feature['create_time'] = train_feature.user_id_hash.map(create_time)
train_feature.create_time = train_feature.create_time.fillna(train_feature['create_time'].value_counts().index[0])
train_feature['session_per_day'] = train_feature.session_count.div(train_feature.create_time)
train_feature['purchase_per_day'] = train_feature.purchase_counts.div(train_feature.create_time)

In [ ]:
train_feature = train_feature.drop(['active_date','create_time'], axis=1)

In [ ]:
train_feature.purchase_per_day  = train_feature.purchase_per_day .fillna(train_feature['purchase_per_day'].value_counts().index[0])

In [ ]:
# the day interval between the latest session day and pred day
# useful
df_train['start_date'] = pd.to_datetime(df_train['start_date'])
df_train ['pred_time'] = '2018-12-1'
df_train ['pred_time'] = pd.to_datetime(df_train ['pred_time'])
df_train ['date_interval'] = df_train['pred_time']-df_train['start_date']
df_train['last_session_time'] = df_train.date_interval.apply(lambda x: int(str(x).split(' ')[0]))

In [ ]:
df_train = df_train.drop(['pred_time','date_interval'], axis=1)

In [ ]:
train_feature['last_session_time'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').last_session_time.min())

In [ ]:
train_feature.head()

### join events info to get more features

In [28]:
file_name = 'events.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
event = pd.read_csv(obj['Body'])
event.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [29]:
purchase_event = event[event['event']=='8']

In [30]:
last_purchase_time = purchase_event.groupby('session_id', as_index=False).event_timestamp.max().rename(columns={'event_timestamp':'last_purchase_time'})
df_event = purchase_event.groupby('session_id',as_index=False).app_id.count().rename(columns={'app_id':'purchase_counts'})
purchase_time = purchase_event.groupby('session_id',as_index=False).event_timestamp.median().rename(columns={'event_timestamp':'purchase_time'})
total_amount = purchase_event.groupby('session_id', as_index=False).event_value.sum().rename(columns={'event_value':'total_amount'})

In [31]:
# get session level purchase count and purchase time info
df_event2 = pd.merge(df_event, purchase_time, how='left', on=['session_id'])
df_event2 = pd.merge(df_event2, last_purchase_time, how='left', on=['session_id'])
df_event2 = pd.merge(df_event2, total_amount, how='left', on=['session_id'])

In [32]:
df_event2.head()

,session_id,purchase_counts,purchase_time,last_purchase_time,total_amount
0,44097414271980,1,1543345213863,1543345213863,13.993
1,139239025219926,1,1543896256185,1543896256185,3.493
2,198661066907208,1,1543274061363,1543274061363,3.493
3,277838121971337,1,1540754187045,1540754187045,3.493
4,330581615202681,1,1541774849854,1541774849854,1.393


In [33]:
# based on session train dataframe join event train, thus we only get info
df_train2 = pd.merge(df_train,df_event2, how='left', on=['session_id'])

In [ ]:
# user purchase count
train_feature['purchase_counts'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').purchase_counts.sum())

In [ ]:
df_train2['purchase_time'] = pd.to_datetime(df_train2.purchase_time + df_train2.timezone_offset, unit='ms')

In [85]:
df_train2['purchase_dow'] = df_train2.purchase_time.dt.dayofweek
df_train2['purchase_hour'] = df_train2.purchase_time.dt.hour

In [94]:
# most frequent purchase dow / hour
train_feature['most_frequent_purchase_dow'] = train_feature.user_id_hash.map(df_train2[df_train2.purchase_counts>0].groupby('user_id_hash').purchase_dow.agg(lambda x: x.value_counts().index[0]))
train_feature.most_frequent_purchase_dow = train_feature.most_frequent_purchase_dow.fillna(7)

In [ ]:
train_feature['most_frequent_purchase_hour'] = train_feature.user_id_hash.map(df_train2[df_train2.purchase_counts>0].groupby('user_id_hash').purchase_hour.agg(lambda x: x.value_counts().index[0]))
train_feature.most_frequent_purchase_hour = train_feature.most_frequent_purchase_hour.fillna(24)

In [ ]:
train_feature['purchase_perc'] = train_feature.purchase_counts/test_feature.session_count

In [97]:
train_feature['total_amount']= train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').total_amount.sum())

In [103]:
df_train2.date_interval.max()

Timedelta('60 days 23:12:10.195000')

In [106]:
# the day interval between the latest purchase day and pred day
# useful
df_train2 ['pred_time'] = '2018-12-1'
df_train2 ['pred_time'] = pd.to_datetime(df_train2 ['pred_time'])
df_train2 ['date_interval'] = df_train2['pred_time']-df_train2['purchase_time']
df_train2.date_interval = df_train2.date_interval.fillna('100 days')
df_train2['last_purchase_time'] = df_train2.date_interval.apply(lambda x: int(str(x).split(' ')[0]))

In [107]:
train_feature['last_purchase_time'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').last_purchase_time.min())

In [109]:
train_feature['purchase_counts'] = train_feature['purchase_counts'].fillna(0)
train_feature['total_amount'] = train_feature['total_amount'].fillna(0)

In [ ]:
train_feature['cost_per_purchase'] = (train_feature.total_amount+1).div((train_feature.purchase_counts+1))

### geographic

* most frequent country/region/city

* mean encoding

In [44]:
df_train2.region.nunique()

754

In [45]:
df_train2.city.nunique()

30998

In [56]:
df_train2.country = df_train2.country.fillna(df_train2['country'].value_counts().index[0])
df_train2.region = df_train2.region.fillna(df_train2['region'].value_counts().index[0])
df_train2.city = df_train2.city.fillna(df_train2['city'].value_counts().index[0])

In [41]:
train_feature['most_freq_country'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').country.agg(lambda x: x.value_counts().index[0]))
train_feature['most_freq_region'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').region.agg(lambda x: x.value_counts().index[0]))
train_feature['most_freq_city'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').city.agg(lambda x: x.value_counts().index[0]))

In [57]:
# no much use
country_mean_purchase_perc = df_train2.groupby('country').purchase_counts.sum()/df_train2.country.value_counts()
train_feature['country_mean_purchase_perc'] = train_feature["most_freq_country"].map(country_mean_purchase_perc)
train_feature.country_mean_purchase_perc = train_feature.country_mean_purchase_perc.fillna(0)

### device info

In [50]:
df_train2.os_name = ['iOS' if x=='iPhone OS' else x for x in df_train2['os_name']]

In [55]:
df_train2.os_name = df_train2.os_name.fillna(df_train2['os_name'].value_counts().index[0])
train_feature['most_freq_os'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').os_name.agg(lambda x: x.value_counts().index[0]))

In [60]:
# no much use
os_mean_purchase_perc = df_train2.groupby('os_name').purchase_counts.sum()/df_train2.os_name.value_counts()
train_feature['os_mean_purchase_perc'] = train_feature["most_freq_os"].map(os_mean_purchase_perc)
train_feature.os_mean_purchase_perc = train_feature.os_mean_purchase_perc.fillna(0)

### Label encoding

* Use label encoding encode categorical columns

In [ ]:
# fillna by most frequentist value
cols = list(train_feature.columns)
for col in cols:
    train_feature[col] = train_feature[col].fillna(train_feature[col].value_counts().index[0])

In [ ]:
import numpy as np
cols = list(train_feature.columns)
for col in cols:
    train_feature[col].replace(np.inf, 0, inplace=True)

In [62]:
from sklearn.preprocessing import LabelEncoder
cols = ['most_freq_country', 'most_freq_region', 'most_freq_city', 'most_freq_os']
for col in cols:
    encoder = LabelEncoder()
    train_feature[col] = encoder.fit_transform(train_feature[col])

In [66]:
train_feature.head()

,user_id_hash,most_freq_dow,most_freq_hour,median_create_session_time,session_count,session_durations,mean_session_durations,mean_session_interval,most_freq_country,most_freq_region,...,total_amount,session_per_active_day,purchase_per_active_day,session_per_day,purchase_per_day,user_entropy,last_session_time,last_purchase_time,country_mean_purchase_perc,os_mean_purchase_perc
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,7,36.0,46,938183565,2.039529e+07,0.782609,162,0,...,3.493,2.555556,0.055556,0.851852,0.018519,3.828641,1,19,0.005269,0.028374
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,5,23,0.0,3,6074816,2.024939e+06,0.000000,69,281,...,0.000,1.500000,0.000000,3.000000,0.000000,1.098612,20,100,0.060754,0.078831
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,2,22,0.0,1,0,0.000000e+00,0.000000,210,299,...,0.000,1.000000,0.000000,0.000000,0.000000,0.000000,45,100,0.068370,0.078831
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,3,15,2.0,1,596967,5.969670e+05,2.000000,94,162,...,0.000,0.500000,0.000000,0.250000,0.000000,0.693147,9,100,0.004529,0.028374
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,2,2,0.5,1,97482,9.748200e+04,0.500000,11,494,...,0.000,0.500000,0.000000,1.000000,0.000000,0.693147,38,100,0.067344,0.028374


In [97]:
train_feature.to_csv('feature_eng/train_feature2.csv', index=False)

### Combine train features and labels

In [71]:
bucket_name = 'advanced-ml-project' # Add your bucket name
file_name = 'sessions_new.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
session = pd.read_csv(obj['Body'])
session.head()

,Unnamed: 0,session_id,timezone,timezone_offset,previous_sessions_duration,is_user_first_session,is_session,country,region,city,...,locale,os_name,session_index,device_id,user_id_hash,start_dow,start_hour,start_date,create_date,create_session_time
0,0,5558845121177764917,Asia/Manila,28800000.0,25837591,False,True,PH,00,makati,...,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,1,2018-11-15,2018-10-07,39
1,1,2201961907282901522,Asia/Manila,28800000.0,35050130,False,True,PH,00,quezon city,...,en_GB,Android OS,47,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,9,2018-12-02,2018-10-07,56
2,2,18781111175537580,Asia/Manila,28800000.0,11343848,False,True,PH,00,makati,...,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,7,2018-10-11,2018-10-07,4
3,3,1477540082628742048,Asia/Manila,28800000.0,13499724,False,True,PH,11,davao city,...,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,19,2018-10-21,2018-10-07,14
4,4,8184875317380844086,Asia/Manila,28800000.0,32788010,False,True,PH,00,makati,...,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,7,2018-11-20,2018-10-07,44


In [91]:
# use more user(creates during pred time) to train
# seems no much use
train_list = pd.DataFrame(session.user_id_hash.unique(), columns=['user_id_hash'])

In [92]:
train_feature2 = pd.merge(train_list, train_feature, how='left', on=['user_id_hash'])

In [93]:
# fillna by most frequentist value
cols = list(train_feature2.columns)
for col in cols:
    train_feature2[col] = train_feature2[col].fillna(train_feature2[col].value_counts().index[0])

In [94]:
import numpy as np
cols = list(train_feature2.columns)
for col in cols:
    train_feature2[col].replace(np.inf, 0, inplace=True)

In [68]:
bucket_name = 'advanced-ml-project' # Add your bucket name
file_name = 'train_label.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
train_label = pd.read_csv(obj['Body'])
train_label.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0.0,0.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0.0,0.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0.0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0.0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0.0,0.0


In [125]:
train_dataset = pd.merge(train_feature, train_label, how='left', on='user_id_hash')
train_dataset.head()

,user_id_hash,most_freq_dow,most_freq_hour,median_create_session_time,session_count,session_durations,mean_session_durations,mean_session_interval,most_freq_country,most_freq_region,...,purchase_per_active_day,session_per_day,purchase_per_day,user_entropy,last_session_time,last_purchase_time,country_mean_purchase_perc,os_mean_purchase_perc,user_purchase_binary_7_days,user_purchase_binary_14_days
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,7,36.0,46,938183565,2.039529e+07,0.782609,162,0,...,0.055556,0.851852,0.018519,3.828641,1,19,0.005269,0.028374,0.0,0.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,5,23,0.0,3,6074816,2.024939e+06,0.000000,69,281,...,0.000000,3.000000,0.000000,1.098612,20,100,0.060754,0.078831,0.0,0.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,2,22,0.0,1,0,0.000000e+00,0.000000,210,299,...,0.000000,0.000000,0.000000,0.000000,45,100,0.068370,0.078831,0.0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,3,15,2.0,1,596967,5.969670e+05,2.000000,94,162,...,0.000000,0.250000,0.000000,0.693147,9,100,0.004529,0.028374,0.0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,2,2,0.5,1,97482,9.748200e+04,0.500000,11,494,...,0.000000,1.000000,0.000000,0.693147,38,100,0.067344,0.028374,0.0,0.0


In [126]:
train_dataset.to_csv('feature_eng/train_dataset2.csv', index=False)

In [116]:
len(train_feature)

619706

In [ ]:
train_dataset2 = pd.merge(train_feature2, train_label, how='left', on='user_id_hash')
train_dataset2.head()

In [127]:
train_dataset2.to_csv('feature_eng/train_dataset3.csv', index=False)